# Answering business questions using SQL

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

# Overview of the data

In [2]:
%%sql
-- List tables and views
SELECT name, type
  FROM sqlite_master
 WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Selecting albums to purchase

The Chinook record store has just signed a deal with a new record label, and we've been tasked with selecting the first three albums that will be added to the store, from the list of four shown below. All four albums are by artists that don't have any tracks in the store right now.

Artist name | Genre
:--- | :---
Regal | Hip-Hop
Red Tone | Punk
Meteor and the Girls | Pop
Slim Jim Bites | Blues

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

In [3]:
%%sql
-- List the sales volume for each genre
  WITH tracks_sold AS
       (
       SELECT g.name genre,
              COUNT(t.track_id) n_tracks
         FROM invoice i
              INNER JOIN invoice_line il
              ON il.invoice_id = i.invoice_id
              INNER JOIN track t
              ON t.track_id = il.track_id
              INNER JOIN genre g
              ON g.genre_id = t.genre_id
        WHERE i.billing_country = 'USA'
        GROUP BY 1
       )
SELECT genre,
       n_tracks,
       ROUND(n_tracks / CAST((SELECT SUM(n_tracks)
                                FROM tracks_sold
                             ) AS FLOAT), 2) percentage
  FROM tracks_sold
 ORDER BY n_tracks DESC
 LIMIT 10;

 * sqlite:///chinook.db
Done.


genre,n_tracks,percentage
Rock,561,0.53
Alternative & Punk,130,0.12
Metal,124,0.12
R&B/Soul,53,0.05
Blues,36,0.03
Alternative,35,0.03
Latin,22,0.02
Pop,22,0.02
Hip Hop/Rap,20,0.02
Jazz,14,0.01


From the four genres listed in the table above, the three that sell the most tracks in the USA are Punk, Pop and Blues. Hence, we would recommend selecting the albums from the following artists:

* Red Tone (Punk)
* Meteor and the Girls (Blues)
* Slim Jim Bites (Pop)

From the three genres mentioned before, Blues is the one that sells most (12% of sales), but the best selling genre in the USA is Rock (53% of sales). We would recommend trying to negotiate a rock album with the new record label.

## Analyzing employee sales performance

In [4]:
%%sql
SELECT e.first_name || ' ' || e.last_name employee_name,
       ROUND(SUM(i.total), 1) total_sales,
       e.hire_date,
       ROUND(SUM(i.total) / MAX(julianday(i.invoice_date) - julianday(e.hire_date)) * 365, 1) sales_per_year
  FROM invoice i
       INNER JOIN customer c
       ON c.customer_id = i.customer_id
       INNER JOIN employee e
       ON e.employee_id = c.support_rep_id
 WHERE e.title = 'Sales Support Agent'
 GROUP BY 1
 ORDER BY 2 DESC;

 * sqlite:///chinook.db
Done.


employee_name,total_sales,hire_date,sales_per_year
Jane Peacock,1731.5,2017-04-01 00:00:00,461.7
Margaret Park,1584.0,2017-05-03 00:00:00,432.8
Steve Johnson,1393.9,2017-10-17 00:00:00,438.6


The top employee, Jane Peacock, sold 24% more than the bottom employee, Steve Johnson. However, if we account for when they were hired and analyse the sales per year, this difference is only 5%.

## Analyzing sales by country

In [5]:
%%sql
  WITH sales AS
       (
        SELECT c.customer_id,
               CASE
               WHEN (
                     SELECT COUNT(DISTINCT customer_id)
                       FROM customer
                      WHERE country = c.country
                    ) = 1 THEN 'Other'
               ELSE country
               END country,
               SUM(i.total) total,
               COUNT(i.total) n_sales
          FROM invoice i
               INNER JOIN customer c
               ON c.customer_id = i.customer_id
         GROUP BY c.customer_id
       )
SELECT country,
       COUNT(DISTINCT customer_id) customers,
       ROUND(SUM(total), 1) total_sales,
       ROUND(SUM(total) / SUM(n_sales), 1) average_sale,
       ROUND(AVG(total), 1) average_sale_per_customer
  FROM (SELECT *,
               CASE
               WHEN country = 'Other' THEN 1
               ELSE 0
               END AS sort
          FROM sales
       )
 GROUP BY country
 ORDER BY sort, total_sales DESC;

 * sqlite:///chinook.db
Done.


country,customers,total_sales,average_sale,average_sale_per_customer
USA,13,1040.5,7.9,80.0
Canada,8,535.6,7.0,66.9
Brazil,5,427.7,7.0,85.5
France,5,389.1,7.8,77.8
Germany,4,334.6,8.2,83.7
Czech Republic,2,273.2,9.1,136.6
United Kingdom,3,245.5,8.8,81.8
Portugal,2,185.1,6.4,92.6
India,2,183.2,8.7,91.6
Other,15,1094.9,7.4,73.0


## Albums vs individual tracks

Management is currently considering changing its purchasing strategy to save money. The strategy being considered is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album. Our task is to find out what percentage of purchases are individual tracks vs whole albums.

In [6]:
%%sql
  WITH invoice_tracks AS
       (
        SELECT i.invoice_id,
               il.track_id,
               t.album_id
          FROM invoice i
               INNER JOIN invoice_line il
               ON il.invoice_id = i.invoice_id
               INNER JOIN track t
               ON t.track_id = il.track_id
       ),
       invoice_album AS
       (
        SELECT invoice_id,
               album_id
          FROM invoice_tracks
         GROUP BY invoice_id
       ),
       is_album AS
       (
        SELECT (
                SELECT track_id
                  FROM invoice_tracks
                 WHERE invoice_id = ia.invoice_id
                EXCEPT
                SELECT track_id
                  FROM track
                 WHERE album_id = ia.album_id
               ) IS NULL
               AND
               (
                SELECT track_id
                  FROM track
                 WHERE album_id = ia.album_id
                EXCEPT
                SELECT track_id
                  FROM invoice_tracks
                 WHERE invoice_id = ia.invoice_id
               ) IS NULL is_album
          FROM invoice_album ia
         GROUP BY invoice_id
       )
SELECT CASE
       WHEN is_album = 0 THEN 'no'
       ELSE 'yes'
       END album_purchase,
       COUNT(*) number_of_invoices,
       round(CAST(COUNT(*) AS FLOAT) / (SELECT COUNT(*) FROM is_album) * 100, 1) percentage
  FROM is_album
 GROUP BY is_album;

 * sqlite:///chinook.db
Done.


album_purchase,number_of_invoices,percentage
no,500,81.4
yes,114,18.6


Since album purchases account for almost one fifth of the revenue, it is not advisable to purchase only the most popular tracks from each album.